In [1]:
from uuid import uuid4
import os
from dotenv import load_dotenv
from databricks.sdk import WorkspaceClient

from ag_vision.drone.ingest import DroneDataIngest

# Load environment variables
load_dotenv(".env")

os.environ.pop('DATABRICKS_HOST', None)
os.environ.pop('DATABRICKS_TOKEN', None)

w = WorkspaceClient(profile='tnau')

In [2]:
# Change the following to the location where the local data is stored.
parent_folder_path = '/Users/danielwilliams/Documents/Field Data/drone_upload_test'
raw_image_dir = f'{parent_folder_path}/raw_flight_data/'
files = os.listdir(raw_image_dir)
files = [raw_image_dir + x for x in files]
files

['/Users/danielwilliams/Documents/Field Data/drone_upload_test/raw_flight_data/DJI_20241124144351_0235_MS_R.TIF',
 '/Users/danielwilliams/Documents/Field Data/drone_upload_test/raw_flight_data/DJI_20241124144459_0269_MS_R.TIF',
 '/Users/danielwilliams/Documents/Field Data/drone_upload_test/raw_flight_data/DJI_20241124144457_0268_MS_RE.TIF',
 '/Users/danielwilliams/Documents/Field Data/drone_upload_test/raw_flight_data/DJI_202411301311_002_TNAU-VBN-30112024-20_PPKRAW.bin',
 '/Users/danielwilliams/Documents/Field Data/drone_upload_test/raw_flight_data/DJI_20241124144453_0266_MS_NIR.TIF',
 '/Users/danielwilliams/Documents/Field Data/drone_upload_test/raw_flight_data/DJI_20241124144459_0269_MS_G.TIF',
 '/Users/danielwilliams/Documents/Field Data/drone_upload_test/raw_flight_data/DJI_20241124144453_0266_D.JPG',
 '/Users/danielwilliams/Documents/Field Data/drone_upload_test/raw_flight_data/DJI_20241124144449_0264_MS_RE.TIF',
 '/Users/danielwilliams/Documents/Field Data/drone_upload_test/raw_

In [3]:
flight_metadata = {
    "id": str(uuid4()),
    "name": "corn_health_assessment_flight",  # this will be the mission name in the folder path
    "task": "drone_phenotyping",
    "location": {
        "site": "TNAU-Coimbatore".lower(),  # Try to keep these standard ORG-Site EX CIAT-Cali, or CIAT-Arusha....
        "field": "West_Field_03".lower(),
        # what is the name of the field that the trial was run on? The field and location can be the same name if there is no difference.
        "location": "Section_B".lower()
        # The location corresponds to a specific field book. If data is in EBS match the location name.
    },
    "trialProperties": {
        "name": "BETS"  # What is the name of the trial, a trial usually has multiple locations.
    },
    "drone_acquisition_properties": {
        'date': '11/25/2024',
        'drone_make': "DJI",
        'drone_model': "Phantom Mavic 4 Enterprise",
        'camera_make': "Sony",
        'camera_model': "UCM-R",
        'groundControlPoints': True,
        'reflectancePanels': True,
        'reflectancePanelType': 'Micasense',
        'flightHeight': 90.0,
        'horizontalOverlapPercentage': 75.0,
        'verticalOverlapPercentage': 75.0,
        'gpsQuality': "RTK",
        'multispecChannels': ["Red", "Green", "Blue", "NIR"]
    },
    "agronomic_properties": {
        "crop_type": "corn",  # Required
        "growth_stage": "VT",  # optional
        "soil_color": None,
        "weed_pressure": None,
        "irrigation_level": None,
        "tillage_type": None,
        "fertilizer_level": None
    }
}

In [4]:
# This points to the aps1-prod-tnau-fg workspace _YOU SHOULD NOT NEED TO CHANGE THIS.
workspace_bucket = '/Volumes/aps1_prod_tnau_fg_catalog_1336582592012881/tier1_raw/data'

ingest = DroneDataIngest(platform='local',  # DONT CHANGE THIS
                         cloud_bucket=workspace_bucket,
                         cloud_client=w,
                         ingest_df=None,  # This will be set LATER
                         flight_date='11/25/2024',
                         plot_boundary_key=f"{parent_folder_path}/plot_boundary.geojson",
                         # must be geojson but name does not matter
                         gcp_key=f"{parent_folder_path}/gcp_details.geojson",
                         # must be geojson but name does not matter
                         orthomosaic_key=f"{parent_folder_path}/30112024_ORTHO.tif",
                         dem_key=f"{parent_folder_path}/30112024_DSM.tif",
                         flight_metadata_key=f"{parent_folder_path}/flight_details.json")  # If this does not exist then this just tells the code were to save it so it can be uploaded.

In [5]:
ingest.load_metadata_from_dict(metadata_dict=flight_metadata)

# season is needed to generate the mission dir, This has lots of Validation and will throw assert errors.
ingest.add_season_code_to_metadata(year=2025,
                                   country='IND',
                                   crop='maize',
                                   time_of_year='spring')

# This is the main dir where all the data will be stored.
ingest.generate_drone_mission_dir_path()

In [6]:
# This saves the metadata locally do it can be uploaded.
ingest.save_flight_metadata_to_json_local()

In [7]:
# This will be a list of all the raw files from the flight eg: nav, bin, tif, and jpg files.
ingest.generate_ingest_df(file_list=files)

In [8]:
# This tells us if it's an image or a flight data file
ingest.ingest_df['file_type'] = ingest.ingest_df['img_ext'].apply(
    lambda x: 'raw_image' if x.lower() in ['.jpeg', '.tif', '.tiff', '.jpg'] else 'flight_data')

In [9]:
# generates the new path names where to save it in databricks
ingest.generate_raw_image_dst_path_name()

In [10]:
ingest.ingest_df.head()

,src_path,file_name,img_ext,file_type,dst_path
0,/Users/danielwilliams/Documents/Field Data/dro...,DJI_20241124144351_0235_MS_R.TIF,.TIF,raw_image,/Volumes/aps1_prod_tnau_fg_catalog_13365825920...
1,/Users/danielwilliams/Documents/Field Data/dro...,DJI_20241124144459_0269_MS_R.TIF,.TIF,raw_image,/Volumes/aps1_prod_tnau_fg_catalog_13365825920...
2,/Users/danielwilliams/Documents/Field Data/dro...,DJI_20241124144457_0268_MS_RE.TIF,.TIF,raw_image,/Volumes/aps1_prod_tnau_fg_catalog_13365825920...
3,/Users/danielwilliams/Documents/Field Data/dro...,DJI_202411301311_002_TNAU-VBN-30112024-20_PPKR...,.bin,flight_data,/Volumes/aps1_prod_tnau_fg_catalog_13365825920...
4,/Users/danielwilliams/Documents/Field Data/dro...,DJI_20241124144453_0266_MS_NIR.TIF,.TIF,raw_image,/Volumes/aps1_prod_tnau_fg_catalog_13365825920...


In [11]:
# Upload the metadata to databricks
ingest.upload_metadata_to_db()

Uploading: 100%|██████████| 1.65k/1.65k [00:02<00:00, 638B/s]  


In [12]:
# uploads the plot boundaries to databricks
ingest.upload_plot_boundary_to_db()

Saving to /Volumes/aps1_prod_tnau_fg_catalog_1336582592012881/tier1_raw/data/tnau-coimbatore/bets/2025:ind:maize:spring/west_field_03/section_b/drone/corn_health_assessment_flight/field_data/plot_boundary.geojson


Uploading: 100%|██████████| 190k/190k [00:01<00:00, 114kB/s]


In [13]:
# Uploads the ground control points to databricks
ingest.upload_gcp_to_db()

Saving to /Volumes/aps1_prod_tnau_fg_catalog_1336582592012881/tier1_raw/data/tnau-coimbatore/bets/2025:ind:maize:spring/west_field_03/section_b/drone/corn_health_assessment_flight/field_data/ground_control_points.geojson


Uploading: 100%|██████████| 1.28k/1.28k [00:00<00:00, 2.07kB/s]


In [14]:
# uploads all the raw flight data to databricks
ingest.upload_raw_flight_data_to_db()

0it [00:00, ?it/s]
Uploading:   0%|          | 0.00/10.1M [00:00<?, ?B/s]
Uploading:   2%|▏         | 160k/9.62M [00:00<00:18, 535kB/s]
Uploading:   4%|▍         | 416k/9.62M [00:00<00:14, 652kB/s]
Uploading:   8%|▊         | 784k/9.62M [00:01<00:11, 785kB/s]
Uploading:  14%|█▍        | 1.34M/9.62M [00:01<00:07, 1.20MB/s]
Uploading:  23%|██▎       | 2.20M/9.62M [00:01<00:03, 2.27MB/s]
Uploading:  27%|██▋       | 2.58M/9.62M [00:01<00:03, 2.03MB/s]
Uploading:  34%|███▍      | 3.25M/9.62M [00:01<00:02, 2.76MB/s]
Uploading:  38%|███▊      | 3.64M/9.62M [00:02<00:02, 2.49MB/s]
Uploading:  41%|████▏     | 3.97M/9.62M [00:02<00:02, 2.60MB/s]
Uploading:  45%|████▍     | 4.28M/9.62M [00:02<00:02, 2.55MB/s]
Uploading:  50%|█████     | 4.84M/9.62M [00:02<00:01, 2.79MB/s]
Uploading:  55%|█████▍    | 5.28M/9.62M [00:02<00:01, 3.13MB/s]
Uploading:  60%|█████▉    | 5.77M/9.62M [00:02<00:01, 3.27MB/s]
Uploading:  66%|██████▋   | 6.38M/9.62M [00:02<00:00, 3.85MB/s]
Uploading:  71%|███████   | 6.83M/9.

In [ ]:
# If you have a dem generated you can upload it so we dont need to reprocess.
ingest.upload_dem_to_db(method='agisoft',  # the method used to generate the dem.
                        dem_date='11/25/2024',  # the date the dem was generated.
                        file_name='dem.tif')

Saving to /Volumes/aps1_prod_tnau_fg_catalog_1336582592012881/tier1_raw/data/tnau-coimbatore/bets/2025:ind:maize:spring/west_field_03/section_b/drone/corn_health_assessment_flight/2024-11-25/dem/agisoft_2024-11-25/dem.tif


Uploading:  30%|██▉       | 440M/1.45G [00:41<00:53, 20.4MB/s] 

In [ ]:
# f"{drone_flight_path}/orthomosaic/{method}_{date}/{image_name}"
ingest.upload_orthomosaic_to_db(method='agisoft',  # the method used to generate the ortho.
                                ortho_date='11/25/2024',  # the date the ortho was generated.
                                file_name='ms.tif')  # if RGB then you can name it rgb.tif